# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [7]:
DATA_FOLDER = './Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

import glob
import natsort
import pandas as pd
from dateutil.parser import parse

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Data field names
Column names and the names of the variables we want to extract slightly vary between countries. Hence, first we define the column names of the _date_ column, _variable_ and _national total_ columns as well as the names of the fields for each country.

In [294]:
# Write your answer here
# Dataframe importing 
# Colums: country, month, averagy, new cases, death

# column names and interesting variables for each country
col_date = {"guinea" : "Date",
            "liberia": "Date",
            "sl" : "date"} # column name of the date column

col_name={"guinea" : "Description",
          "liberia": "Variable",
          "sl" : "variable"} # column name of the indicator variables

# TODO total vs new deaths / cases
col_total = {"guinea":"Totals",
             "liberia":"National",
             "sl": "National"} # column name where the national totals per day are stored
# TODO sum counties if no national data availble
# TODO Make sure you handle all the different expressions for new cases and deaths that are used in the reports.
col_counties={"guinea":["Conakry","Gueckedou","Macenta","Dabola","Kissidougou","Dinguiraye","Telimele","Boffa","Kouroussa","Siguiri","Pita","Nzerekore","Yomou","Dubreka","Forecariah","Kerouane","Coyah","Dalaba","Beyla","Kindia"],
              "liberia":[""],
              "sl":[""]}

var_death =  {"guinea" : ["New deaths registered today (confirmed)","New deaths registered"],
             "liberia": ["Newly reported deaths"],# "Total death/s in confirmed cases"
             "sl":["etc_new_deaths"]}

var_death_total =  {"guinea" : ["Total deaths of confirmed"],
             "liberia": ["Total death/s in confirmed cases"],# "Total death/s in confirmed cases"
             "sl":["death_confirmed"]}

var_cases = {"guinea" : ["New cases of confirmed"],
             "liberia": ["New case/s (confirmed)"],
             "sl":["cum_confirmed"]}


### Aggregate data frames
Now, we loop over all files and import them, applying necessary clean-up operations such as type conversions, comma stripping etc. For each file (day), we extract one row with the corresponding values and merge them in `df`.

In [324]:
dfs = [] # dataframes for each day
for c in ['guinea','liberia','sl']:
    for file in sorted(glob.glob(DATA_FOLDER+"/ebola/"+c+"_data/*.csv")):
        df = pd.read_csv(file,index_col=None)
        df = df.fillna(0) # Fill NA values
        
        # Get Data
        # Dates
        date = pd.to_datetime(df[1:2][col_date[c]]) # dates
        
        # New cases
        df_cases = df[df[col_name[c]].isin(var_cases[c])] # extract only relevant rows
        df_cases = df_cases[[col_date[c],col_name[c],col_total[c]]] # extract only relevant columns (date - name - total)
        df_cases[0:1][col_total[c]].values[0] = str(df_cases[0:1][col_total[c]].values[0]).replace(',','') # strip commas for 1000s
        df_cases[col_total[c]] = df_cases[col_total[c]].astype(int)     # convert to int (e.g., 10.0 or 10,0)
        cases = int(df_cases[0:1][col_total[c]]) # cases

        # New deaths
        df_deaths = df[df[col_name[c]].isin(var_death[c])] # extract only relevant rows
        df_deaths = df_deaths[[col_date[c],col_name[c],col_total[c]]] # extract only relevant columns (date - name - total)
        deaths = int(df_deaths[0:1][col_total[c]]) # deaths 
        
        # TODO Total Deaths
        df_deaths_total = df[df[col_name[c]].isin(var_death_total[c])] # extract only relevant rows
        df_deaths_total = df_deaths_total[[col_date[c],col_name[c],col_total[c]]] # extract only relevant columns (date - name - total)
        deaths_total = int(df_deaths_total[0:1][col_total[c]]) # deaths 
        # TODO Total Deaths by County
        
        # new dataframe from data (one row)
        df = pd.DataFrame({"Country":c,"Date":date,"Cases":cases,"New Deaths":deaths,"Total Deaths":deaths_total},index=None)
        dfs.append(df)
        
for i in range(len(dfs)-1): # concatenate all dataframes (rows)
    dfs[0] = pd.concat([dfs[0],dfs[i+1]])
df=dfs[0]

df=df.sort_values(by=['Country','Date']) # sort by country and date
df=df.set_index(['Country','Date']) # new index
df.head()

Cases  New Deaths  Total Deaths
Country Date                                       
guinea  2014-08-04      4           2           228
        2014-08-26     10           5           292
        2014-08-27     10           2           294
        2014-08-30      9           5           324
        2014-08-31     29           3           337

### Data checking

Let's check the data whether the index is unique and whether there are still NaN values

In [325]:
# Data checking
print("Index unique:")
print(df.index.is_unique) # True
for field in ("New Deaths","Total Deaths","Cases"):
    print(field+":")
    print(df[df[field].isnull()].size) # get NaN colums (all 0)

Index unique:
True
New Deaths:
0
Total Deaths:
0
Cases:
0


### Aggregate by month

Now, we can regroup the data by month and sums each column by the month. Then, we rename the date column as month column 

In [327]:
# Aggregate by month
df=df.reset_index()
df_month = pd.DataFrame(df.groupby([df.Country,df.Date.dt.month]).sum()).reset_index()
df_month = df_month.rename(columns={'Date':'Month'})
df_month = df_month.set_index(['Country','Month'])
df_month

index  Cases  New Deaths  Total Deaths
Country Month                                        
guinea  8         10     62          17          1475
        9        200    208          57          7217
        10        21      6          15           562
liberia 6        175     15          14           122
        7        374     20          47           476
        8        396     49         209          1762
        9       1452    148         865         14562
        10      2125     34         701          6188
        11      1575     39         202             0
        12      1053  17355           0             0
sl      8       2630  15565           0          5877
        9       4524  43555           8         12856
        10      5166  86149          99         25353
        11      4389  94092          12         23581
        12      1110  31463          11          6574

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [12]:
# Write your answer here

# Read in Excel files, concatenate them and add file name (MID1...9) for later merging
i=1
MID = pd.read_excel(DATA_FOLDER+'/microbiome/MID'+str(i)+'.xls', sheetname='Sheet 1', header=None,index_col=0)
MID.columns=['Count']
MID['BARCODE'] = "MID1" # add column with file name
for i in range(2,10):
    MID_tmp=pd.read_excel(DATA_FOLDER+'/microbiome/MID'+str(i)+'.xls', sheetname='Sheet 1', header=None,index_col=0)
    MID_tmp.columns=['Count']
    MID_tmp['BARCODE'] = "MID"+str(i)  # add column with file name
    MID=pd.concat([MID, MID_tmp], axis=0)
# read metadata

MID.index.name="Species"
MID.head()
metadata = pd.read_excel(DATA_FOLDER+'/microbiome/metadata.xls', sheetname='Sheet1',index_col=0)

# merge metadata and files
MID_merged = pd.merge(metadata, MID, left_index=True, right_on='BARCODE')
# print(MID_merged.index.is_unique) # True
MID_merged=MID_merged.fillna("unknown")
MID_merged=MID_merged.reset_index().set_index(['BARCODE','Species']) # new index
MID_merged.head()

GROUP  \
BARCODE Species                                                                  
MID1    Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  EXTRACTION CONTROL   
        Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  EXTRACTION CONTROL   
        Archaea "Crenarchaeota" Thermoprotei Sulfolobal...  EXTRACTION CONTROL   
        Archaea "Crenarchaeota" Thermoprotei Thermoprot...  EXTRACTION CONTROL   
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...  EXTRACTION CONTROL   

                                                             SAMPLE  Count  
BARCODE Species                                                             
MID1    Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  unknown      7  
        Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  unknown      2  
        Archaea "Crenarchaeota" Thermoprotei Sulfolobal...  unknown      3  
        Archaea "Crenarchaeota" Thermoprotei Thermoprot...  unknown      3  
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...  unknown      7

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here